# Homework 1 (Due: February 27, 2023)

## 1. Fitting a model to data

1. Prepare an artificial data set composed of $t$ = \[0.0, 0.1, 0.2,..., 200\] and $y = 0.8e^{-c_1 t}+0.2 e^{-c_2 t}$ where $c_1$ = 0.1 and $c_2$ = 0.01. Then, use [`scipy.optimize.curve_fit`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html) to find the best fit of our "model" of the functional form $y = A \exp(-a t)$ where $A$ and $a$ are constant parameters.
2. Add gaussian noise ([`numpy.random.randn`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.randn.html)) to the $y$ data and see how the model fitting is affected depending on the amplitude of the noise. Also, compare the results with a model with two exponential terms, $y = A_1 \exp(-a_1 t) + A_2 \exp(-a_2 t)$. How would you determine which model is better given the goodness of fit and number of parameters?
3. Load the data file `ipsc_acc_17711000.dat` (Use [`numpy.loadtxt`](https://numpy.org/doc/stable/reference/generated/numpy.loadtxt.html)). Note that the first and second column are time in millisecond and a time series derived from experimental data, respectively) and try to fit the sum-of-exponentials models to this data. Choose your best model/fit and describe how you validate the result.

## 2. Inspecting membrane properties via current clamp

Following an example in our NEURON Tutorial 1, create an axon simulation
with the following specifications:

- Diameter (`diam`) = 1 um and length (`L`) = 1000 um,
- Capacitance per area (`cm`) = 1 uF/cm2,
- Passive mechanism (`pas`) with the conductance per area = 1×10$^{-4}$ S/cm2. (**Hint**: the conductance variable for this mechanism is `axon.g_pas`)
- A current clamp electrode (`ic`) near one end (`x`≈0) that injects 50 pA for a sufficiently long time (>200 ms).

1. What are the theoretical membrane time constant (τ) and space constant (λ)?
2. Run the simulation and measure the membrane time constant from the simulated data. How does the measured time constant compare with the theoretical value?
3. What is the electrotonic length of this axon?
4. **(Fig. 2.4)** By changing the length (`L`), plot the voltage attenuation curve in Fig. 2.4 in the textbook. Note that the unconnected cylinder ends in NEURON are sealed, and you will reproduce the "Sealed end" case. Discuss implications in membrane potential dynamics in short versus long dendrites.


## 3. Fig. 2.5 and 2.8B

Using the simulation in Quesion 2, plot the sealed end case of Fig. 2.5 in the textbook by following the steps below. 

1. **(I-V curve)** Try a few different values for the injected current from -50 pA to 50 pA, record the steady state membrane voltages, and plot the current vs membrane voltage graph. What is the input resistance?
2. **(Fig. 2.4)** Draw the cable length curve versus input resistance curve as Fig. 2.5 in the textbook. Please make sure that the cable has the sufficiently large `nseg` to get a quasi-smooth curve. Again, you should be able to get only one curves since cables in NEURON have sealed-ends.
3. **(Fig. 2.8)** Set the length sufficiently long and plot how the voltage changes in time during current injection at different locations as in Fig. 2.8.